# 02 - Analysis
* Load the depth data created during preprocessing
* Load weights from the provided PFRA JSON file
* Estimate depth, upper CI, and lower CI for each pixel for each return period

In [ ]:
import ffrd_data_utils as fdu
import xarray as xr
import json

In [ ]:
from dask.distributed import Client

client = Client()
client

In [ ]:
DEPTH_ZARR = "./data/depth.zarr"
WEIGHTS = "./data/Addison_M03_H04_Weights.json"
RETURN_PERIODS = [10, 50, 100, 500]
AEPS = [1/return_period for return_period in RETURN_PERIODS]
WSEL_RECHUNKED_ZARR = "./data/wsel_rechunked.zarr"

In [ ]:
# load the pre-processed depth dataset from the previous notebook
depth_ds = xr.open_zarr(DEPTH_ZARR)
depth_ds

In [ ]:
# load weights for each depth raster
with open(WEIGHTS, 'r') as weights_file:
    weights = list(json.load(weights_file).values())

In [ ]:
# for each return period:
# 1. estimate the depth at each pixel
# 2. compute the upper and lower CI for the depth at each pixel
depth_quantiles_ds = fdu.depth_quantiles(depth_ds.depth, AEPS, spatial_ref=depth_ds.spatial_ref, weights=weights)
depth_quantiles_ds

In [ ]:
# write results to disk
depth_quantiles_ds.to_zarr('./data/depth_quantiles.zarr', mode='w')

In [ ]:
# check out the results
depth_quantiles_ds = xr.open_zarr('./data/depth_quantiles.zarr')
depth_quantiles_ds

## Visualization

### Depth

In [ ]:
# depth at AEP == 0.002 (500-yr return period)
depth = depth_quantiles_ds.depth.sel(aep=0.002)
depth.where(depth > 0.0).plot()

In [ ]:
# depth at AEP == 0.1 (10-yr return period)
depth = depth_quantiles_ds.depth.sel(aep=0.1)
depth.where(depth > 0.0).plot()

In [ ]:
# difference in depth between 500-yr and 10-yr return periods
delta = depth_quantiles_ds.depth.sel(aep=0.002) - depth_quantiles_ds.depth.sel(aep=0.1)
delta.plot()

### Confidence Interval

In [ ]:
# 10-year return period
ci_range = depth_quantiles_ds.upper_ci.sel(aep=0.1) - depth_quantiles_ds.lower_ci.sel(aep=0.1)
ci_range.plot()

In [ ]:
# 500-year return period
ci_range = depth_quantiles_ds.upper_ci.sel(aep=0.002) - depth_quantiles_ds.lower_ci.sel(aep=0.002)
ci_range.plot()

### Water Surface Elevation

In [ ]:
wsel_ds = xr.open_zarr(WSEL_RECHUNKED_ZARR)
terrain_da = wsel_ds.terrain

In [ ]:
# 10-year water surface elevation
(depth_quantiles_ds.depth.sel(aep=0.1) + terrain_da).plot()

In [ ]:
# 500-year water surface elevation
(depth_quantiles_ds.depth.sel(aep=0.002) + terrain_da).plot()